# Process PM measurements and Event Logs 

This notebook processes user data to create analyst-friendly output files.  

**Event log processing**: This loads users' raw event logs and outputs separate *notification log*, *annotation log*, and *spike log* files.  Each row records their respective event types.  These porcessed files also include additional columns explicitly listing additional timestamp characteristics (hour of day, day of week, weekday vs. weekend, etc), to make the data more amenable to filtering and aggregating.

**PM log processing**: much the same as event log processing.  This adds Additional columns to break the timestamp into more analyst-friendly parameters for quick analysis.


## External libraries

In [11]:
import pandas as pd
import numpy as np
import datetime
import pytz

## External Code

In [12]:
# %load functions.py

#////////////////////////////
#
#  LOOK UP TABLE FUNCTIONS
#
#////////////////////////////

def LUT(deployment,monitor,timestamp):
    return {
        'deployment_001': dep1LUT(monitor,timestamp),
        'deployment_003': dep3LUT(monitor,timestamp),
        'deployment_007': dep7LUT(monitor,timestamp),
        'deployment_008': dep8LUT(monitor,timestamp),
        'deployment_009': dep9LUT(monitor,timestamp),
        'deployment_012': dep12LUT(monitor,timestamp),
    }[deployment] 
def getDeploymentFromParticipantID(pid):
    return{
    'P2':'deployment_001',
    'P1':'deployment_003',
    'P3':'deployment_008',
    'P4':'deployment_009',
    'P5':'deployment_012',
    'P6':'deployment_007',
    }[pid]
def getMonitorsfromDeployment(dep):
    return{
        'deployment_001': ['monitorb001','monitorb002','monitorb003','monitorb015','monitorb020','monitorb021'],
        'deployment_003': ['monitorb004','monitorb005','monitorb006','monitorb012','monitorb013','monitorb014'],
        'deployment_007': ['monitorb003','monitorb016','monitorb018','monitorb022','monitorb023','monitorb024'],
        'deployment_008': ['monitorb007','monitorb009','monitorb015','monitorb016','monitorb017','monitorb019','monitorb021','monitorb110_'],
        'deployment_009': ['monitorb001','monitorb002','monitorb014','monitorb018','monitorb019','monitorb020'],
        'deployment_012': ['monitorb010','monitorb011','monitorb017','monitorb022','monitorb023','monitorb024'],
    }[dep]
def deploymentLUT(dep):
    return{
    'deployment_001':'P2',
    'deployment_003':'P1',
    'deployment_008':'P3',
    'deployment_009':'P4',
    'deployment_012':'P5',
    'deployment_007':'P6',
    }[dep]

def getMonitorLocationsFromDeployment(dep):
    return{
        'deployment_001': ['living room','bedroom','outside'],
        'deployment_003': ['bedroom','downstairs','outside'],
        'deployment_007': ['outside','sons room','kitchen'],
        'deployment_008': ['bedroom','kitchen','downstairs','backyard'],
        'deployment_009': ['bedroom','front room','outside'],
        'deployment_012': ['bedroom','living room','front room','back yard','front yard', 'downstairs'],
    }[dep]

    
def dep1LUT(monitor,timestamp):
    
    # monitors in deployment001: 1,2,3,15,17,20,21
    
    if monitor == 'monitorb001':
        return 'bedroom'
    
    elif monitor == 'monitorb002':
         return 'living room'
        
    elif monitor == 'monitorb003':
        return 'outside'
    
    elif monitor == 'monitorb015':
        return 'bedroom'
    
    elif monitor == 'monitorb017':
        return 'outside?'
    
    elif monitor == 'monitorb020':
        return 'living room'
    
    elif monitor == 'monitorb021':
        return 'outside'
    
    else:
        return "unknown"   
def dep3LUT(monitor,timestamp):
    
    # monitors in deployment003: 4,5,6,12,13,14
    
    if monitor == 'monitorb004':
        return 'bedroom'
    
    elif monitor == 'monitorb005':
         return 'downstairs'
        
    elif monitor == 'monitorb006':
        return 'outside'
    
    elif monitor == 'monitorb012':
        return 'bedroom'
    
    elif monitor == 'monitorb013':
        return 'downstairs'
    
    elif monitor == 'monitorb014':
        return 'outside'
    else:
        return "unknown"
def dep7LUT(monitor,timestamp):
    
    # monitors in deployment007: 3,16,18,24
    
    if monitor == 'monitorb003':
        return 'outside'
    
    elif monitor == 'monitorb016':
         return 'sons room'
        
    elif monitor == 'monitorb018':
        return 'kitchen'
    
    elif monitor == 'monitorb022':
        return 'sons room'
    
    elif monitor == 'monitorb023':
        return 'kitchen'
    
    elif monitor == 'monitorb024':
        return 'sons room'
    
    else:
        return "unknown"   
def dep8LUT(monitor,timestamp):
    
    # monitors in deployment003: 7,9,15,16,17,19,21,110
    
    if monitor == 'monitorb007':
        return 'bedroom?'
    
    elif monitor == 'monitorb009':
         return 'kitchen'
        
    elif monitor == 'monitorb015':
        return 'downstairs'
    
    elif monitor == 'monitorb016':
        return '???'
    
    elif monitor == 'monitorb017':
        return 'backyard'
    
    elif monitor == 'monitorb019':
        return 'bedroom'
    
    elif monitor == 'monitorb021':
        return 'kitchen'
    
    elif monitor == 'monitorb110_':
        return 'backyard'
    
    else:
        return "unknown"  
def dep9LUT(monitor,timestamp):
    
    # monitors in deployment009: 1,2,14,18,19,20
    
    if monitor == 'monitorb001':
        return 'bedroom'
    
    elif monitor == 'monitorb002':
         return 'front room'
        
    elif monitor == 'monitorb014':
        return 'outside'
    
    elif monitor == 'monitorb018':
        return 'bedroom'
    
    elif monitor == 'monitorb019':
        return 'front room'
    
    elif monitor == 'monitorb020':
        return 'outside'
    else:
        return "unknown"  
def dep12LUT(monitor,timestamp):
    
    # monitors in deployment012: 10,11,17,22,23,24
    
    if monitor == 'monitorb010':
        
        if pd.to_datetime(timestamp) < pd.to_datetime('2018-05-11 21:00:00'):
            return 'bedroom'
        else:
            return 'living room'

    elif monitor == 'monitorb011':
        return 'living room'
    
    elif monitor == 'monitorb017':
         return 'downstairs'
    
    elif monitor == 'monitorb022':
        return 'downstairs'
    
    elif monitor == 'monitorb023':
        
        if pd.to_datetime(timestamp) < pd.to_datetime ('2018-05-11 21:00:00'):
            return 'back yard'
        else:
            return 'front yard'
    
    elif monitor == 'monitorb024':
        return 'front yard'
    
    else:
        return "unknown"
def monthLabel(num):
    return{
        1:"January",
        2:"February",
        3:"March",
        4:"April",
        5:"May",
        6:"June",
        7:"July",
        8:"August",
        9:"Septemeber",
        10:"October",
        11:"November",
        12:"December"
    }[num]
def resolutionLabel(x):
    return {
        'M': "Month",
        'D': "Day",
        'H': "Hour",
        'W': "Week"
    }[x]
def dayOfWeekLabel(num):
    return{
        0:'Monday',
        1:'Tuesday',
        2:'Wednesday',
        3:'Thursday',
        4:'Friday',
        5:'Saturday',
        6:'Sunday',
    }[num]


#////////////////////////////
#
#  CLASSIFICATION FUNCTIONS
#
#////////////////////////////

def classifyIndoorsOrOutdoors(location):
    if location not in ['outside','front yard', 'back yard','backyard']:
        return 'indoors'
    else:
        return 'outdoors'
    
def getSeason(month):

    season = (month%12+3)//3
   
    return {
        1: 'winter',
        2: 'spring',
        3: 'summer',
        4: 'fall'
    }[season]

def phaseOfDay(hour):

    phase = "morning"
    
    if hour >=0 and hour <6:
        phase = "night"
    elif hour >=6 and hour <12:
        phase = "morning"
    elif hour >=12 and hour <18:
        phase = "afternoon"
    elif hour >=18 and hour <=21:
        phase = "evening"
    elif hour >=21 and hour <24:
        phase = "night"
    
    return phase
    
def classifyWeekdayOrWeekend(day):
    if day not in ['Friday','Saturday','Sunday']:
        return 'weekday'
    else:
        return 'weekend'
def classifyDatetimeSlices(df):
    
    #df.index = pd.to_datetime(df.event_time)
    df.index = pd.to_datetime(df.event_time_mst)
    df['year'] = df.index.year
    df['month_num'] = df.index.month
    df['month'] = df['month_num'].apply(monthLabel)
  
    #df['day'] = df.index.day
    df['hour'] = df.index.hour
    df['day_phase'] = df['hour'].apply(phaseOfDay)
   
    df['day of week'] = df.index.dayofweek
    df['weekday'] = df['day of week'].apply(dayOfWeekLabel)
    df['week_phase'] = df['weekday'].apply(classifyWeekdayOrWeekend)
    
    df['week number'] = df.index.week
    df['season'] = df['month_num'].apply(getSeason)
    
    
    
    # clean up unneccessary colums
    delCol = ["month_num",
              "day of week"]
    df.drop(delCol, axis=1,inplace=True)
    
    return df

def classifyMonitorLocations(df):
    df['location'] = df.apply(lambda row: LUT(row['home_id'], row['entity_id'], row['time']), axis=1)
    return df

def addDataFrameClassifiers(df):
    # This function takes your AQ dataframe and generates additional columns for classifying
    # measurement types

    # create column to categorize measurements as indoors or outdoors
   
    df = classifyMonitorLocations(df);
    df['location_type'] = df['location'].apply(classifyIndoorsOrOutdoors)
   
    #### Add more classification functions for categorizing AQ measurements

    #df = classifyDatetimeSlices(df)

    return df

#/////////////////////////////////
#
#  DATAFRAME EDITING FUNCTIONS
#
#/////////////////////////////////

def writeLocations(row):
    return LUT(row['home_id'],row['entity_id'], row['time'])

def removeMonitorSuffix(row):
    return row[0:11]  # remove  '_small'suffix

def generateIAQOAQDifference(df, res, measure):
    # compute average indoor PM
    indoorData = df[df['location type']=='indoors']
    averageIndoorPm = indoorData.resample(res)[measure].mean()

    #compute average outdoor PM
    outdoorData = df[df['location type']=='outdoors']
    averageOutdoorPm = outdoorData.resample(res)[measure].mean()

    indoorOutdoorDifference = averageIndoorPm - averageOutdoorPm

    IAQOAQDiff = indoorOutdoorDifference.to_frame()
    IAQOAQDiff = timestampToColumns(IAQOAQDiff)

    return IAQOAQDiff

def processPMLog(participant):

    suffix = "_analyst_set_pm.csv"
    filepath =  participant+suffix
    print("Loading: " +filepath)
    
    # The encoding is necessary for P6, most likely some wonky filetype or formatting issue 
    df = pd.read_csv(filepath, encoding = "ISO-8859-1");
    
    #What is this participant's deployment ID?
    deployment = getDeploymentFromParticipantID(participant)

    ## Adding columns to improve process-ability

    # set time to datetime object & convert to mst
    df['time'] = pd.to_datetime(df['time'])
    df["time"] = df["time"].dt.tz_localize('US/Mountain', ambiguous=True)
    
    # set time_mst as index to make the following easier...
    df.set_index(['time'], inplace=True)
    
    # add necessary columns
    df['year'] = df.index.year
    df['month_num'] = df.index.month
    df['month'] = df['month_num'].apply(monthLabel)
  
    #df['day'] = df.index.day
    df['hour'] = df.index.hour
    df['day_phase'] = df['hour'].apply(phaseOfDay)
   
    df['day of week'] = df.index.dayofweek
    df['weekday'] = df['day of week'].apply(dayOfWeekLabel)
    df['week_phase'] = df['weekday'].apply(classifyWeekdayOrWeekend)
    
    df['week number'] = df.index.week
    df['season'] = df['month_num'].apply(getSeason)
    
    
    
    # clean up unneccessary colums
    delCol = ["month_num",
              "day of week"]
    df.drop(delCol, axis=1,inplace=True)

    return df


def processEventLog(participant):
    
    #load PX_events_raw.csv files locally.  See Jimmy for copies.
    suffix = "_events_raw.csv"
    filepath = participant+suffix 
    print("Loading: " +filepath)
    
    # The encoding is necessary for P6, most likely some wonky filetype or formatting issue 
    data = pd.read_csv(filepath,encoding = "ISO-8859-1");
    
    # remove extra columns from raw file
    df = removeRawColumns(data)
    
    #What is this participant's deployment ID?
    deployment = getDeploymentFromParticipantID(participant)
    
    #because the P5 and P6 files have different timestamp columns
    
    if participant == "P5" or participant == "P6":
        print("Deployment is : " + deployment)
        # Fix the event time and insert time columns
        df['event_time'] = df['event_time'].apply(lambda row: formatTimeColumn(row))
        df['insert_time'] = df['insert_time'].apply(lambda row: formatTimeColumn(row))
        #print(df.head())
      
    #  set timestamp columns to atetimeobjects, localize to UTC, and covert to MST
    df['event_time'] = pd.to_datetime(df['event_time'])
    df["event_time_mst"] = df["event_time"].dt.tz_localize('UTC').dt.tz_convert('America/Denver')
    
    df['insert_time'] = pd.to_datetime(df['insert_time'])
    df["insert_time_mst"] = df["insert_time"].dt.tz_localize('UTC').dt.tz_convert('America/Denver')


    # split into separate logs
    annotation_log = df[df["event_type"]=="annotation"]
    spike_log = df[df["event_type"]=="peak"]
    notification_log = df[df["event_type"]=="notification"]
    
    # Add locations to spike_log
    spike_log['location'] = spike_log.apply(lambda row:LUT(deployment, row["peak_monitor"], row["event_time"]), axis=1 )
    
    #format notification_log to add a 'location' column for highlighting where the spike occured
    notification_log["location"] = notification_log.apply(lambda row: findLocationInTextString(deployment,row["notification_text"]), axis=1)
    
    # remove log-specific columns (those which aren't necessary for respective log files)
    annotation_log = pruneAnnotationLogColumns(annotation_log)
    spike_log = pruneSpikeLogColumns(spike_log)
    notification_log = pruneNotificationLogColumns(notification_log)

    #package dataframes in a dict
    output =  {
        "annotations":   annotation_log,
        "spikes":        spike_log,
        "notifications": notification_log
    }
    
    return output
def formatTimeColumn(date):
    #print(date.replace('"',"").replace("T"," ").replace("Z",""))
    date = date.replace('"',"").replace("T"," ").replace("Z","")
    return date

def removeRawColumns(df):
    delCol = ["_id",
              "peak_id",
              "notification_destination",
              "sms_phone_number",
              "google_home_id",
              "notification_id",
              "notification_trigger",
              "peak_id",
              "technician_monitor"]
    
    df.drop(delCol, axis=1,inplace=True)
    
    return df

def pruneAnnotationLogColumns(df):
    delCol = ["notification_text",
             "peak_value",
             "peak_monitor",
             "deployment",
             "event_type"]
    df.drop(delCol, axis=1,inplace=True)
    return df
def pruneSpikeLogColumns(df):
    delCol = ["annotation_source",
             "annotation_text",
              "notification_text",
             "deployment",
             "peak_monitor",     # we've used this to identify the monitor location, so we don't need it anymore
              "insert_time",     # this column only tells us when it was added to mongo
              "insert_time_mst", # don't care about this
              "event_type",] 
    df.drop(delCol, axis=1,inplace=True)
    return df

def pruneNotificationLogColumns(df):
    delCol = ["annotation_source",
             "annotation_text",
             "peak_value",
             "peak_monitor", 
             "event_type",
             "deployment",
             "insert_time",
             "insert_time_mst"]   # the entry is made the moment the notification goes out, so this column is the same
    df.drop(delCol, axis=1,inplace=True)
    return df
def formatSpikeLog(df):
    deployment = df["deployment"].unique()
    
    print(deployment)

def getLocation(row):
    return LUT(row["deployment"],row["peak_monitor"],row["event_time"])

def findLocationInTextString(dep, txt):
    
    place = "unknown"
    
    #get list of all monitor placements per deployment
    locations = getMonitorLocationsFromDeployment(dep);
    
    # inefficient, but see whether location is in the notification string for each place
    for location in locations:
        if(txt.find(location)>0):
            place = location;
        
    return place
    

## Process event Logs

In [13]:
participants = ["P5"]

for person in participants:

    pid = person
    
    event_dict = processEventLog(pid)

    log_names = ["annotations","spikes","notifications"]

    print(pid +":")
    for log in log_names:
        logfile = event_dict[log]
        logfile = classifyDatetimeSlices(logfile)
        filename = pid + "_" + log + "_log.csv"
        logfile.to_csv(filename, index=False)

        print("\t"+log + " log file written.") 

Loading: P5_events_raw.csv
Deployment is : deployment_012
P5:
	annotations log file written.
	spikes log file written.
	notifications log file written.


/var/folders/_t/9wrf8_zj063fsrjdhb8h98640000gn/T/ipykernel_1855/1677914207.py:456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spike_log['location'] = spike_log.apply(lambda row:LUT(deployment, row["peak_monitor"], row["event_time"]), axis=1 )
/var/folders/_t/9wrf8_zj063fsrjdhb8h98640000gn/T/ipykernel_1855/1677914207.py:459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notification_log["location"] = notification_log.apply(lambda row: findLocationInTextString(deployment,row["notification_text"]), axis=1)


## Process PM logs

In [49]:
particiapnts = ["P1","P2","P3","P4","P6","P5"]

for person in particiapnts:

    print(person + ": processing pm log...")
    pm_log = processPMLog(person)
    print("\t...Log processed")
    print("Log head:")
    
    display(pm_log.head())
    
    print("Writing log to file...")
    filename = person + "_pm_log.csv"
    pm_log.to_csv(filename)

    print("\t"+person + " log file written.") 

P1: processing pm log...
Loading: pm_data/P1_analyst_set_pm.csv
	...Log processed
Log head:


entity_id         home_id  value    location  \
time                                                                        
2017-07-17 18:00:18-06:00  monitorb012  deployment_003     53     bedroom   
2017-07-17 18:00:26-06:00  monitorb014  deployment_003    343     outside   
2017-07-17 18:00:37-06:00  monitorb013  deployment_003     88  downstairs   
2017-07-17 18:01:17-06:00  monitorb012  deployment_003     57     bedroom   
2017-07-17 18:01:26-06:00  monitorb014  deployment_003    334     outside   

                          location_type  year month  hour day_phase weekday  \
time                                                                          
2017-07-17 18:00:18-06:00       indoors  2017  July    18   evening  Monday   
2017-07-17 18:00:26-06:00      outdoors  2017  July    18   evening  Monday   
2017-07-17 18:00:37-06:00       indoors  2017  July    18   evening  Monday   
2017-07-17 18:01:17-06:00       indoors  2017  July    18   evening  Monday   
2017-07-17 18:01:26-06:00      outdoors  2017  July    18   evening  Monday   

                          week_phase  week number  season  
time                                                       
2017-07-17 18:00:18-06:00    weekday           29  summer  
2017-07-17 18:00:26-06:00    weekday           29  summer  
2017-07-17 18:00:37-06:00    weekday           29  summer  
2017-07-17 18:01:17-06:00    weekday           29  summer  
2017-07-17 18:01:26-06:00    weekday           29  summer

Writing log to file...
	P1 log file written.
P2: processing pm log...
Loading: pm_data/P2_analyst_set_pm.csv
	...Log processed
Log head:


entity_id         home_id  value     location  \
time                                                                         
2017-07-24 18:00:18-06:00  monitorb001  deployment_001    148      bedroom   
2017-07-24 18:00:55-06:00  monitorb002  deployment_001     74  living room   
2017-07-24 18:01:17-06:00  monitorb001  deployment_001    157      bedroom   
2017-07-24 18:01:54-06:00  monitorb002  deployment_001     66  living room   
2017-07-24 18:02:16-06:00  monitorb001  deployment_001    176      bedroom   

                          location_type  year month  hour day_phase weekday  \
time                                                                          
2017-07-24 18:00:18-06:00       indoors  2017  July    18   evening  Monday   
2017-07-24 18:00:55-06:00       indoors  2017  July    18   evening  Monday   
2017-07-24 18:01:17-06:00       indoors  2017  July    18   evening  Monday   
2017-07-24 18:01:54-06:00       indoors  2017  July    18   evening  Monday   
2017-07-24 18:02:16-06:00       indoors  2017  July    18   evening  Monday   

                          week_phase  week number  season  
time                                                       
2017-07-24 18:00:18-06:00    weekday           30  summer  
2017-07-24 18:00:55-06:00    weekday           30  summer  
2017-07-24 18:01:17-06:00    weekday           30  summer  
2017-07-24 18:01:54-06:00    weekday           30  summer  
2017-07-24 18:02:16-06:00    weekday           30  summer

Writing log to file...
	P2 log file written.
P3: processing pm log...
Loading: pm_data/P3_analyst_set_pm.csv
	...Log processed
Log head:


...

Writing log to file...
	P4 log file written.
P6: processing pm log...
Loading: pm_data/P6_analyst_set_pm.csv
	...Log processed
Log head:


entity_id         home_id  value   location  \
time                                                                       
2018-01-20 17:00:05-07:00  monitorb003  deployment_007    647    outside   
2018-01-20 17:00:50-07:00  monitorb024  deployment_007     79  sons room   
2018-01-20 17:00:53-07:00  monitorb018  deployment_007    127    kitchen   
2018-01-20 17:01:06-07:00  monitorb003  deployment_007    664    outside   
2018-01-20 17:01:51-07:00  monitorb024  deployment_007     82  sons room   

                          location_type  year    month  hour  day_phase  \
time                                                                      
2018-01-20 17:00:05-07:00      outdoors  2018  January    17  afternoon   
2018-01-20 17:00:50-07:00       indoors  2018  January    17  afternoon   
2018-01-20 17:00:53-07:00       indoors  2018  January    17  afternoon   
2018-01-20 17:01:06-07:00      outdoors  2018  January    17  afternoon   
2018-01-20 17:01:51-07:00       indoors  2018  January    17  afternoon   

                            weekday week_phase  week number  season  
time                                                                 
2018-01-20 17:00:05-07:00  Saturday    weekend            3  winter  
2018-01-20 17:00:50-07:00  Saturday    weekend            3  winter  
2018-01-20 17:00:53-07:00  Saturday    weekend            3  winter  
2018-01-20 17:01:06-07:00  Saturday    weekend            3  winter  
2018-01-20 17:01:51-07:00  Saturday    weekend            3  winter

Writing log to file...
	P6 log file written.
P5: processing pm log...
Loading: pm_data/P5_analyst_set_pm.csv
	...Log processed
Log head:


entity_id         home_id  value    location  \
time                                                                        
2018-03-03 17:00:08-07:00  monitorb023  deployment_012    707   back yard   
2018-03-03 17:00:15-07:00  monitorb022  deployment_012    400  downstairs   
2018-03-03 17:00:15-07:00  monitorb010  deployment_012    195     bedroom   
2018-03-03 17:01:09-07:00  monitorb023  deployment_012    668   back yard   
2018-03-03 17:01:16-07:00  monitorb022  deployment_012    401  downstairs   

                          location_type  year  month  hour  day_phase  \
time                                                                    
2018-03-03 17:00:08-07:00      outdoors  2018  March    17  afternoon   
2018-03-03 17:00:15-07:00       indoors  2018  March    17  afternoon   
2018-03-03 17:00:15-07:00       indoors  2018  March    17  afternoon   
2018-03-03 17:01:09-07:00      outdoors  2018  March    17  afternoon   
2018-03-03 17:01:16-07:00       indoors  2018  March    17  afternoon   

                            weekday week_phase  week number  season  
time                                                                 
2018-03-03 17:00:08-07:00  Saturday    weekend            9  spring  
2018-03-03 17:00:15-07:00  Saturday    weekend            9  spring  
2018-03-03 17:00:15-07:00  Saturday    weekend            9  spring  
2018-03-03 17:01:09-07:00  Saturday    weekend            9  spring  
2018-03-03 17:01:16-07:00  Saturday    weekend            9  spring

Writing log to file...
	P5 log file written.
